<a href="https://colab.research.google.com/github/mdmmn378/Augmentor/blob/master/crnn_nb_x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Settings

In [0]:
!pip install pytorch_lightning
!pip install -U torchvision

In [1]:
!wget https://medusa.fit.vutbr.cz/traffic/download/512/

--2019-09-20 16:42:00--  https://medusa.fit.vutbr.cz/traffic/download/512/
Resolving medusa.fit.vutbr.cz (medusa.fit.vutbr.cz)... 147.229.8.49, 2001:67c:1220:808::93e5:831
Connecting to medusa.fit.vutbr.cz (medusa.fit.vutbr.cz)|147.229.8.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1908808499 (1.8G) [application/octet-stream]
Saving to: ‘index.html’

index.html          100%[===================>]   1.78G   110MB/s    in 17s     

2019-09-20 16:42:18 (109 MB/s) - ‘index.html’ saved [1908808499/1908808499]



In [0]:
!mv index.html ./data.zip

In [0]:
%%capture
!unzip data.zip

# Import Modules

In [0]:
%pdb

Automatic pdb calling has been turned ON


In [0]:
from sklearn.utils import shuffle

In [0]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import collections

import torch
import torch.nn as nn
from torch.autograd import Variable # depricated
from torch.utils.data import DataLoader,Dataset
from torch.nn import functional as F
import torchvision.transforms as transforms
from torch import optim

from IPython.core.debugger import set_trace
from IPython.display import clear_output

from fastprogress import master_bar, progress_bar

import pytorch_lightning as pl
from test_tube import Experiment
from pytorch_lightning import  Trainer
from pytorch_lightning.callbacks import ModelCheckpoint


%matplotlib inline

In [0]:
import time

# Utils

In [0]:
class strLabelConverter(object):
    def __init__(self, alphabet, ignore_case=True):
        self._ignore_case = ignore_case
        if self._ignore_case:
            alphabet = alphabet.lower()
        self.alphabet = alphabet + '~'  # for `-1` index

        self.dict = {}
        for i, char in enumerate(alphabet):
            # NOTE: 0 is reserved for 'blank' required by wrap_ctc
            self.dict[char] = i + 1

    def encode(self, text):
        if isinstance(text, str):
            text = [
                self.dict[char.lower() if self._ignore_case else char]
                for char in text
            ]
            length = [len(text)]
        elif isinstance(text, collections.Iterable):
            length = [len(s) for s in text]
            text = ''.join(text)
            text, _ = self.encode(text)
        return (torch.IntTensor(text), torch.IntTensor(length))

    def decode(self, t, length, raw=False):
#         set_trace()
        if length.numel() == 1:
            length = length[0]
            assert t.numel() == length, "text with length: {} does not match declared length: {}".format(t.numel(), length)
            if raw:
                return ''.join([self.alphabet[i - 1] for i in t])
            else:
                char_list = []
                for i in range(length):
                    if t[i] != 0 and (not (i > 0 and t[i - 1] == t[i])):
                        char_list.append(self.alphabet[t[i] - 1])
                return ''.join(char_list)
        else:
            # batch mode
            assert t.numel() == length.sum(), "texts with length: {} does not match declared length: {}".format(t.numel(), length.sum())
            texts = []
            index = 0
            for i in range(length.numel()):
                l = length[i]
                texts.append(
                    self.decode(
                        t[index:index + l], torch.IntTensor([l]), raw=raw))
                index += l
            return texts


# Augmentation and Preprocessing

In [0]:
# df = pd.read_csv('./data/trainVal.csv')
# img = Image.open(os.path.join('data',df['image_path'][0]))
# img

In [0]:
class DataSet(Dataset):
    def __init__(self,root='./',dfn='trainVal.csv',img_height=32,img_width=96,alphabet='0123456789abcdefghijklmnopqrstuvwxyz'):
        self.root  = root
        self.df = pd.read_csv(os.path.join(root,dfn))
        self.tensorize = transforms.ToTensor()
        self.resize = transforms.Resize((img_height,img_width))
        self.alphabet = alphabet
        self.converter = strLabelConverter(self.alphabet)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,index):
        loc = self.df['image_path'][index]
        label = self.df['lp'][index]
        img = Image.open(os.path.join(self.root,loc)).convert('L')
        img = self.resize(img)
        img = self.tensorize(img)
        label = self.converter.encode(label)
        return img,label
        
        

# Data Handler

In [0]:
def spliter(dataset,ratio):
    ln = len(dataset)
    ln1 = int(ln*ratio)
    ln2 = ln-ln1
    split1,split2 = torch.utils.data.random_split(dataset,[ln1,ln2])
    return split1,split2

In [0]:
train_dataset = DataSet()
train_dataset,val_dataset = spliter(train_dataset,0.8)


In [0]:
def collate_fn(batch):
    batch_size = len(batch)
    images = [i[0] for i in batch]
    images = torch.stack(images)
    
    targets  = [i[1][0] for i in batch]
    targets = torch.cat([*targets],dim=0).cuda()
    
    target_lens = torch.cuda.IntTensor([len(i[1][0]) for i in batch])
    return images,batch_size,targets,target_lens
    

In [0]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True,collate_fn=collate_fn)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True,collate_fn=collate_fn)

In [0]:
# # # %%capture
for info in val_loader:
    print(info)
    break

(tensor([[[[0.0863, 0.0824, 0.0941,  ..., 0.1725, 0.1922, 0.2118],
          [0.0784, 0.0745, 0.0745,  ..., 0.2353, 0.2588, 0.2784],
          [0.1059, 0.0902, 0.0784,  ..., 0.2706, 0.2863, 0.3059],
          ...,
          [0.0588, 0.0588, 0.0588,  ..., 0.0745, 0.0745, 0.0784],
          [0.0667, 0.0667, 0.0667,  ..., 0.0667, 0.0706, 0.0706],
          [0.0667, 0.0667, 0.0667,  ..., 0.0627, 0.0667, 0.0667]]],


        [[[0.2235, 0.2235, 0.2314,  ..., 0.2745, 0.2745, 0.2745],
          [0.2118, 0.2196, 0.2275,  ..., 0.2627, 0.2667, 0.2627],
          [0.2039, 0.2118, 0.2196,  ..., 0.2549, 0.2549, 0.2549],
          ...,
          [0.1569, 0.1569, 0.1569,  ..., 0.1255, 0.1216, 0.1216],
          [0.1490, 0.1529, 0.1529,  ..., 0.1137, 0.1137, 0.1176],
          [0.1412, 0.1412, 0.1451,  ..., 0.1098, 0.1137, 0.1176]]],


        [[[0.8706, 0.8667, 0.8667,  ..., 0.8431, 0.8706, 0.8863],
          [0.8627, 0.8627, 0.8588,  ..., 0.8000, 0.8039, 0.7961],
          [0.8431, 0.8392, 0.8353,  .

# Model

In [0]:
class BidirectionalLSTM(nn.Module):

    def __init__(self, nIn, nHidden, nOut):
        super(BidirectionalLSTM, self).__init__()

        self.rnn = nn.LSTM(nIn, nHidden, bidirectional=True)
        self.embedding = nn.Linear(nHidden * 2, nOut)

    def forward(self, input):
#         set_trace()
        recurrent, _ = self.rnn(input)
        T, b, h = recurrent.size()
        t_rec = recurrent.view(T * b, h)

        output = self.embedding(t_rec)  # [T * b, nOut]
        output = output.view(T, b, -1)

        return output



In [0]:

class CRNN(nn.Module):

    def __init__(self, imgH, nc, nclass, nh, n_rnn=2, leakyRelu=False):
        super(CRNN, self).__init__()
        assert imgH % 16 == 0, 'imgH has to be a multiple of 16'

        ks = [3, 3, 3, 3, 3, 3, 2]
        ps = [1, 1, 1, 1, 1, 1, 0]
        ss = [1, 1, 1, 1, 1, 1, 1]
        nm = [64, 128, 256, 256, 512, 512, 512]

        cnn = nn.Sequential()

        def convRelu(i, batchNormalization=False):
            nIn = nc if i == 0 else nm[i - 1]
            nOut = nm[i]
            cnn.add_module('conv{0}'.format(i),
                           nn.Conv2d(nIn, nOut, ks[i], ss[i], ps[i]))
            if batchNormalization:
                cnn.add_module('batchnorm{0}'.format(i), nn.BatchNorm2d(nOut))
            if leakyRelu:
                cnn.add_module('relu{0}'.format(i),
                               nn.LeakyReLU(0.2, inplace=True))
            else:
                cnn.add_module('relu{0}'.format(i), nn.ReLU(True))

        convRelu(0)
        cnn.add_module('pooling{0}'.format(0), nn.MaxPool2d(2, 2))  # 64x16x64
        convRelu(1)
        cnn.add_module('pooling{0}'.format(1), nn.MaxPool2d(2, 2))  # 128x8x32
        convRelu(2, True)
        convRelu(3)
        cnn.add_module('pooling{0}'.format(2),
                       nn.MaxPool2d((2, 2), (2, 1), (0, 1)))  # 256x4x16
        convRelu(4, True)
        convRelu(5)
        cnn.add_module('pooling{0}'.format(3),
                       nn.MaxPool2d((2, 2), (2, 1), (0, 1)))  # 512x2x16
        convRelu(6, True)  # 512x1x16

        self.cnn = cnn
        self.rnn = nn.Sequential(
            BidirectionalLSTM(512, nh, nh),
            BidirectionalLSTM(nh, nh, nclass))

    def forward(self, input):
#         set_trace()
        conv = self.cnn(input)
        b, c, h, w = conv.size()
        assert h == 1, "the height of conv must be 1"
        conv = conv.squeeze(2)
        conv = conv.permute(2, 0, 1)  # [w, b, c]

        # rnn features
        output = self.rnn(conv)

        return output


# Experimental Setup

In [0]:
class NumberPlateDetector(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = CRNN(32, 1, 37, 256)
#         for param in self.nn_model.parameters():
#             param.requires_grad = False
    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_nb):
        x, batch,y,ys = batch
        y_hat = self.forward(x)
#         set_trace()
        xs = torch.empty([y_hat.shape[1]],dtype=torch.int,device='cuda')
        xs.fill_(y_hat.shape[0])
        criterion = torch.nn.CTCLoss().to('cuda')
        loss = criterion(y_hat.to('cpu'),y.to('cpu'),xs.to('cpu'),ys.to('cpu'))
        return {'loss': loss}

#     def validation_step(self, batch, batch_nb):
#         x, y = batch
#         y_hat = self.forward(x)
#         return {'val_loss': F.cross_entropy(y_hat, y)}

#     def validation_end(self, outputs):
#         avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#         return {'avg_val_loss':avg_loss}

#     def configure_optimizers(self):
#         return [torch.optim.Adam(self.parameters(), lr=0.02),torch.optim.Adam(self.parameters(), lr=0.02)]
    
#     def on_batch_start(self):
#         do something when the batch starts
#      def on_tng_metrics(self, metrics):
#         pass
#     def update_tng_log_metrics(self, logs):
#         return logs


#     def optimizer_step(self, current_epoch, batch_nb, optimizer, optimizer_i):
#         set_trace()
#         optimizer.step()
#         optimizer.zero_grad()
#         if optimizer_i == 0:
#             if batch_nb % 2 == 0 :
#                 optimizer.step()
#                 optimizer.zero_grad()

#         if optimizer_i == 1:
#             if batch_nb % 4 == 0 :
#                 optimizer.step()
#                 optimizer.zero_grad()  

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(),lr=0.5e-2)
        scheduler = optim.lr_scheduler.CyclicLR(optimizer,max_lr=0.5e-2,base_lr=5e-4,cycle_momentum=False)
#         return [optimizer], [scheduler]                
        return optimizer

    @pl.data_loader
    def tng_dataloader(self):
        return train_loader

#     @pl.data_loader
#     def val_dataloader(self):
#         return val_loader

#     @pl.data_loader
#     def test_dataloader(self):
#         return DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor()), batch_size=64)

# Train and Evaluation

In [0]:
model = NumberPlateDetector()
exp = Experiment(save_dir='./weights')

In [0]:
trainer = Trainer(max_nb_epochs=300,experiment=exp,show_progress_bar=True,gpus=[0],checkpoint_callback=None)

VISIBLE GPUS: '0'
gpu available: True, used: True


In [0]:
model.train()
#trainer.optimizers[0].lr = 0.0001
trainer.fit(model)

  0%|          | 1/4559 [00:00<06:13, 12.22it/s, batch_nb=0, epoch=2, gpu=0, loss=0.016, v_nb=0]

                     Name               Type   Params
0                   model               CRNN  8331301
1               model.cnn         Sequential  5551360
2         model.cnn.conv0             Conv2d      640
3         model.cnn.relu0               ReLU        0
4      model.cnn.pooling0          MaxPool2d        0
5         model.cnn.conv1             Conv2d    73856
6         model.cnn.relu1               ReLU        0
7      model.cnn.pooling1          MaxPool2d        0
8         model.cnn.conv2             Conv2d   295168
9    model.cnn.batchnorm2        BatchNorm2d      512
10        model.cnn.relu2               ReLU        0
11        model.cnn.conv3             Conv2d   590080
12        model.cnn.relu3               ReLU        0
13     model.cnn.pooling2          MaxPool2d        0
14        model.cnn.conv4             Conv2d  1180160
15   model.cnn.batchnorm4        BatchNorm2d     1024
16        model.cnn.relu4               ReLU        0
17        model.cnn.conv5   

 24%|██▎       | 1076/4559 [01:24<04:31, 12.84it/s, batch_nb=1075, epoch=3, gpu=0, loss=0.018, v_nb=0]

KeyboardInterrupt: ignored

# Visualization

# Saving Model

In [0]:
torch.save(model.state_dict(), './model.ckpt')

In [0]:
# model.load_state_dict(torch.load('./model.ckpt'))

# Other Experiments

In [0]:
img = Image.open('./s01_l01/2_1.png').convert('L')
img = transforms.Resize((32,96))(img)
img = transforms.ToTensor()(img)
img = img.reshape(1,*img.shape).to('cuda')

In [0]:
model.eval()
preds = model(img)

In [0]:
xs = torch.empty([preds.shape[1]],dtype=torch.int).to('cuda')
xs.fill_(preds.shape[0])
_, preds = preds.max(2)
preds = preds.transpose(1, 0).contiguous().view(-1)

In [0]:
converter = strLabelConverter('0123456789abcdefghijklmnopqrstuvwxyz')

In [0]:
converter.decode(preds,xs)

'7c2498'